In [14]:
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from rbbm_src.labelling_func_src.src.lfs import *
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from rbbm_src.labelling_func_src.src.classes import SPAM, HAM, ABSTAIN, lf_input_internal, clean_text
import pandas as pd
import glob

In [15]:
all_filenames = [i for i in glob.glob('/home/opc/chenjie/labelling_explanation/labelling_func_src/src/data/Youtube05-Shakira.csv')]
sentences_df = pd.concat([pd.read_csv(f) for f in all_filenames ])
sentences_df = sentences_df.rename(columns={"Type": "label", "Tweet": "text"})
sentences_df = sentences_df.rename(columns={"CLASS": "label", "CONTENT": "text"})
sentences_df['text'] = sentences_df['text'].apply(lambda s: clean_text(s))

In [16]:
applier = PandasLFApplier(lfs=lattice_lfs)

In [17]:
lattice_lfs

[func: keyword_my,
 func: keyword_shakira,
 func: short_comment,
 func: keyword_shakira_short_comment]

In [18]:
initial_vectors = applier.apply(df=sentences_df, progress_bar=False)
model = LabelModel(cardinality=2, verbose=True)
model.fit(L_train=initial_vectors, n_epochs=500, log_freq=100, seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|                                                                                                                                                                                                                              | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.063]
INFO:root:[100 epochs]: TRAIN:[loss=0.003]
 32%|██████████████████████████████████████████████████████████████████▊                                                                                                                                               | 159/500 [00:00<00:00, 1586.43epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.003]
INFO:root:[300 epochs]: TRAIN:[loss=0.003]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1582.71epoch/s]
INFO:root:Fini

In [19]:
# probs_test= model.predict_proba(L=initial_vectors)
# df_sentences_filtered, probs_test_filtered = filter_unlabeled_dataframe(
#         X=sentences_df, y=probs_test, L=initial_vectors
# )

In [20]:
# df_sentences_filtered

In [21]:
# probs_test= model.predict_proba(L=initial_vectors)
# df_sentences_filtered, probs_test_filtered = filter_unlabeled_dataframe(
#         X=sentences_df, y=probs_test, L=initial_vectors
# )
# # reset df_train to those receive signals
# df_sentences_filtered = df_sentences_filtered.reset_index(drop=True)

# filtered_vectors = applier.apply(df=df_sentences_filtered, progress_bar=False)

In [22]:
sentences_df['model_pred'] = pd.Series(model.predict(L=initial_vectors))

In [23]:
sentences_df

,COMMENT_ID,AUTHOR,DATE,text,label,model_pred
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,nice song,0,0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,i love song,0,0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,2015-05-28T21:00:08.607000,i love song,0,0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,lets make it first female to reach...,0,-1
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,2015-05-28T17:08:29.827000,shakira is best for worldcup,0,0
...,...,...,...,...,...,...
366,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,i love this song for two reasons it is abou...,0,-1
367,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0,0
368,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,shakira u are so wiredo,0,0
369,_2viQ_Qnc685RPw1aSa1tfrIuHXRvAQ2rPT9R06KTqA,Latin Bosch,2013-07-12T22:33:27.916000,shakira is the best dancer,0,0


In [24]:
sentences_df[(sentences_df['label']!=sentences_df['model_pred'])& (sentences_df['model_pred']!=ABSTAIN)][['text','label','model_pred']]

,text,label,model_pred
7,see some more song open google and type shakir...,1,0
38,love this song my soccer team made a cd for o...,0,1
59,my friend sam loves this song,0,1
81,definitley the song for when im not liste...,1,0
125,i loved this song when i was in my teenage years,0,1
132,help shakira s waka waka be the first song...,1,0
139,love this song so much one of my faves xxx,0,1
146,me and my aunt love this song,0,1
147,i subscribed it br,1,0
149,top three shakira songs my choice br br ...,1,0


In [25]:
sentences_df[sentences_df['text'].str.contains(r'woo', na=True)]

,COMMENT_ID,AUTHOR,DATE,text,label,model_pred
27,z12yxbpp3rurin5wc04cf3sqmlrvwlhzgao,Golden Lion,2015-05-26T17:16:06.269000,shakira is my favourite singer wooooo,0,0


In [26]:
sentences_df

,COMMENT_ID,AUTHOR,DATE,text,label,model_pred
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,nice song,0,0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,i love song,0,0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,2015-05-28T21:00:08.607000,i love song,0,0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,lets make it first female to reach...,0,-1
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,2015-05-28T17:08:29.827000,shakira is best for worldcup,0,0
...,...,...,...,...,...,...
366,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,i love this song for two reasons it is abou...,0,-1
367,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0,0
368,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,shakira u are so wiredo,0,0
369,_2viQ_Qnc685RPw1aSa1tfrIuHXRvAQ2rPT9R06KTqA,Latin Bosch,2013-07-12T22:33:27.916000,shakira is the best dancer,0,0


In [44]:
l=[]

In [45]:
for i in range(0,3):
    l.append(sentences_df.sample(1))

In [46]:
# sdf=sentences_df.sample(1)

In [47]:
import random

In [48]:
random.sample(l, 2)

[                             COMMENT_ID         AUTHOR  \
 62  z12uvzpw4tufgf2qw04chd5abu3uvf4rw5o  Eric Gonzalez   
 
                           DATE  \
 62  2015-05-24T15:35:03.844000   
 
                                                  text  label  model_pred  
 62  to help shakira become the first female to hit...      0           0  ,
                                 COMMENT_ID          AUTHOR  \
 141  z125trmyqrbaijwhf04cctobjlq4ixy4gz00k  Mohamed Anhari   
 
                            DATE                    text  label  model_pred  
 141  2015-05-19T23:02:25.583000  best song  go shaki  d      0          -1  ]

In [41]:
sdf.to_dict('records')[0]

{'COMMENT_ID': '_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA',
 'AUTHOR': 'Katie Mettam',
 'DATE': '2013-07-13T13:27:39.441000',
 'text': 'i love this song because we sing it at camp all the time  ',
 'label': 0,
 'model_pred': -1}